### 5-5 활성화 함수 계층 구현하기

이제 신경망을 구성하는 층 각각을 클래스로 구현해보자. 먼저 활성화 함수인 ReLU와 Sigmoid 계층을 구현한다. 

#### 5.5.1ReLU 계층
ReLU는 x가 0보다 크면 x를, 작거나 같으면 0을 출력하는 함수이다. 이때 x에 대한 y의 미분은 1과 0으로 나온다.  
x가 0보다 크면 상류의 값을 그대로 흘려보내고, 반대의 경우는 흘려보내지 않는다. 

In [2]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

Relu 클래스는 mask라는 인스턴스 변수를 가진다. 이는 True / False로 구성된 넘파이 배열로 순전파의 입력으로 들어온 x의 원소값이 0 이하인 인덱스는 True, 그 외는 False로 유지한다.  

In [3]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


Relu의 순전파에서 0보다 작은 값이 입력으로 들어오면 0이 되야 한다.  
mask 값이 True 이면 0보다 작다는 뜻이므로 0을 넣고, 0보다 그면 False 리므로 그대로 남아있는다.  
역전파 당시에는 순전파때 만들어 놓은 mask를 사용해서 상류에서 전파된 dout을 0으로 만든다. 

#### 5.5.2 Sigmoid 계층

시그모이드 식은 다음과 같다.  
y = 1 / 1 + exp(-x)  여기서 곱하기와 더하기 노드이외에도 나누기와 exp 노드가 새로 등장한다.  
곱하고, exp, 더하고, 나눗셈  

1단계)
먼저 /, y = 1/x를 미분하면 -1/x^2, -y^2이 된다. 이는 역전파 당시 상류에서 흘러온 값에 -y^2 (순전파의 출력을 제곱한 후 마이너스를 붙인 값)을 곱해서 하류로 전달한다.  

2단계)
덧셈 노드는 값을 여과 없이 하류로 보낸다.

3단계)
y = exp(x)의 미분은 exp(x)이다. 순전파 당시의 출력을 곱해 하류로 전달한다.

4단계)
곱하기 노드는 순전파 당시의 값을 서로 바꿔서 곱한다.

그럼 최종 출력은 y^2exp(-x)가 되는데 시그모이드는 이처럼 계산 그래프의 중간 과정을 묶어 입력 x와 출력 y로만 계산이 가능하다.  
더 나아가 노드를 그룹화하여 y(1 - y)로 계산할 수도 있다.  

In [ ]:
class Sigmoid():
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.dout) * self.out # 순전파의 출력을 변수 out에 보관했다가 역전파 계산시 사용한다.  
        
        return dx